In [3]:
# !pip install mplfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.6 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import mplfinance as mpf
from typing import Any

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

plt.rcParams['figure.dpi'] == 300
sns.set_style('whitegrid')
%matplotlib inline

First of all, I can't just model one ticker as it would make the model really specialised and hence it won't be able to generalize.

One ticker means:
*   No Generalization
*   No learnable pattern across assets.
*   Just us fitting a fancy curve on some noise lmao

Some places to start?

Model cross-sectional relationships and temporal patterns:
* How does this stock behave relative to others?
* What sectors is it in? What indices move it?
* What macro factors affect it?
* What regimes (bull, bear, volatility spikes) shape it?

Lets start by creating a multi-ticker database